In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=False)

memory.save_context({"input":"Hi!"}, {"output":"오늘 하루 어때?"})

memory.load_memory_variables({})




In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(return_messages=True, k=3)

def add_message(input, output):
    memory.save_context({"input":input}, {"output":output})

add_message(1, 1)
add_message(2, 2)
add_message(3, 3)
add_message(4, 4)

memory.load_memory_variables({})

In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)

def get_history():
    return memory.load_memory_variables({})

add_message("안녕 나는 김진범이라고 해, 한국 의정부라는 도시에 살고 있어", "안녕 만나서 반가워")
add_message("안녕 나는 김진범이라고 해, 한국 의정부라는 도시에 살고 있어2", "안녕 만나서 반가워2")

get_history()

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=50,
    return_messages=True
)

add_message("안녕 나는 김진범이라고 해, 한국 의정부라는 도시에 살고 있어", "안녕 만나서 반가워")
add_message("안녕 나는 김진범이라고 해, 한국 의정부라는 도시에 살고 있어", "안녕 만나서 반가워")

get_history()


In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 도움을 주는 시스템이야"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}")
    ]
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True
)

chain.predict(question="내 이름은 jinbeom이야")

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True
)

examples = [
    {
        "question": "탑건",
        "answer": """
            🛩️👨‍✈️🔥
        """
    }, 
    {
        "question": "대부",
        "answer": """
            👨‍👨‍👦🔫🍝
        """
    }    
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{movie}"),
        ("ai", "{answer}"),
    ]
)
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 영화 제목을 듣고 세개의 이모티콘으로 표현해야해"),
        # ("system", "너는 대화를 도와줘야 해"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{movie}")
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"movie": question})
    memory.save_context(
        {"input":question},
        {"output": result.content}
    )
    print(result)

invoke_chain("신세계")
invoke_chain("앞에서 말한 영화제목은 뭐지?")